# Habitech Clustering for Match Making Feature

This notebook will contain Habitech Clustering form match making feature using N2D Clustering. This notebook will use 33 as initial number of cluster as the total number of genre is 33. Note that this initial number of cluster is still have much room for improvement.

Paper Source:

https://arxiv.org/pdf/1908.05968.pdf

## Import Package

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn import mixture
%matplotlib inline

%pip install umap-learn
import umap

     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 1.2MB 40.0MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.1-cp37-none-any.whl size=76569 sha256=c0decf863f8204c4c0a21c7bcec1104720b1e552efb8cca6ea2e89e1c495278d
  Stored in directory: /root/.cache/pip/wheels/ad/df/d5/a3691296ff779f25cd1cf415a3af954b987fb53111e3392cf4
  Created wheel for pynndescent: filename=pynndescent-0.5.2-cp37-none-any.whl size=51362 sha256=018649d038ddf56bdec7b7130e66de8b6bf89909cc272eef1bf8d7c53cc47e21
  Stored in directory: /root/.cache/pip/wheels/ba/52/4e/4c28d04d144a28f89e2575fb63628df6e6d49b56c5ddd0c74e
Successfully built umap-learn pynndescent


## Load Data

In [2]:
df = pd.read_csv('./final_encoded_dataset.csv')
df

,age,x0_SD,x0_SMP,x0_SMA,x0_Diploma,x0_S1,x0_S2,x0_S3,x0_Profesi,x0_Medical,x0_Science-Geography,x0_Art-Photography,x0_Biography,x0_Business-Finance-Law,x0_Childrens-Books,x0_Computing,x0_Crafts-Hobbies,x0_Crime-Thriller,x0_Dictionaries-Languages,x0_Entertainment,x0_Food-Drink,x0_Graphic-Novels-Anime-Manga,x0_Health,x0_History-Archaeology,x0_Home-Garden,x0_Humour,x0_Mind-Body-Spirit,x0_Natural-History,x0_Personal-Development,x0_Poetry-Drama,x0_Reference,x0_Religion,x0_Romance,x0_Science-Fiction-Fantasy-Horror,x0_Society-Social-Sciences,x0_Sport,x0_Stationery,x0_Teaching-Resources-Education,x0_Technology-Engineering,x0_Teen-Young-Adult,x0_Transport,x0_Travel-Holiday-Guides
0,34,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,34,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,37,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13304,34,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13305,43,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13306,34,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13307,23,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data Analysis (N2D)

### Autoencoder

In [3]:
'''
Autoencoder
  - dimension: d-500-500-2000-c
  - where:
    - d is the dimensionality of the data
    - c is the number of clusters
  - the decoder network is a mirror of the encoder
  - all layers use ReLU activation 
  - the optimizer is Adam
  - train the autoencoder on for 1000 epochs for all datasets
'''

# Autoencoder
def autoencoder_model(data_shape, number_cluster=33):
  ae_input = tf.keras.layers.Input(shape=(data_shape), name='input_layer')

  ae_encode = tf.keras.layers.Dense(units=500, activation='relu', name='encoder_0')(ae_input)
  ae_encode = tf.keras.layers.Dense(units=500, activation='relu', name='encoder_1')(ae_encode)
  ae_encode = tf.keras.layers.Dense(units=2000, activation='relu', name='encoder_2')(ae_encode)
  ae_encode = tf.keras.layers.Dense(units=number_cluster, activation='relu', name='encoder_3')(ae_encode)

  ae_decode = tf.keras.layers.Dense(units=2000, activation='relu', name='decoder_3')(ae_encode)
  ae_decode = tf.keras.layers.Dense(units=500, activation='relu', name='decoder_2')(ae_decode)
  ae_decode = tf.keras.layers.Dense(units=500, activation='relu', name='decoder_1')(ae_decode)
  ae_decode = tf.keras.layers.Dense(units=data_shape, activation='relu', name='decoder_0')(ae_decode)

  model = tf.keras.models.Model(inputs=ae_input, outputs=ae_decode)

  return model

### UMAP

In [4]:
'''
UMAP
  - number of neighbours is 20 (n_neighbors)
  - number of dimensions is the number of clusters (n_components)
  - minimum distance between each point in the manifold is 0
'''
# Manifold learner
def manifold_learner(n_neighbors=20, n_components=33, metric='euclidean', min_dist=0):
  return umap.UMAP(
      n_neighbors=n_neighbors, 
      n_components=n_components, 
      metric=metric, 
      min_dist=min_dist, 
      random_state=0
    )

### GMM

In [5]:
'''
GMM
  - GMM for the final clustering algorithm
  - where each component has its own general covariance matrix and there are c components, where c is the number of clusters
'''
# Clustering Algorithm
def clustering_algorithm(n_components=33):
  return mixture.GaussianMixture(n_components=n_components, random_state=0)

### Build Autoencoder Model

In [6]:
tensor_df = tf.convert_to_tensor(df)
tensor_df

<tf.Tensor: shape=(13309, 42), dtype=float64, numpy=
array([[34.,  0.,  0., ...,  0.,  0.,  0.],
       [34.,  0.,  0., ...,  0.,  0.,  0.],
       [37.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [34.,  0.,  0., ...,  0.,  0.,  0.],
       [23.,  0.,  0., ...,  0.,  0.,  0.],
       [33.,  0.,  0., ...,  0.,  0.,  0.]])>

In [7]:
ae_model = autoencoder_model(tensor_df.shape[-1])
ae_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 42)]              0         
_________________________________________________________________
encoder_0 (Dense)            (None, 500)               21500     
_________________________________________________________________
encoder_1 (Dense)            (None, 500)               250500    
_________________________________________________________________
encoder_2 (Dense)            (None, 2000)              1002000   
_________________________________________________________________
encoder_3 (Dense)            (None, 33)                66033     
_________________________________________________________________
decoder_3 (Dense)            (None, 2000)              68000     
_________________________________________________________________
decoder_2 (Dense)            (None, 500)               100050

In [8]:
ae_model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam())
ae_model.fit(tensor_df, tensor_df, epochs=1000, verbose=1)

Epoch 1/1000
416/416 [==============================] - 4s 3ms/step - loss: 0.5220
Epoch 2/1000
416/416 [==============================] - 1s 3ms/step - loss: 0.0672
Epoch 3/1000
416/416 [==============================] - 1s 3ms/step - loss: 0.0705
Epoch 4/1000
416/416 [==============================] - 1s 3ms/step - loss: 0.0830
Epoch 5/1000
416/416 [==============================] - 1s 2ms/step - loss: 0.0724
Epoch 6/1000
416/416 [==============================] - 1s 2ms/step - loss: 0.0735
Epoch 7/1000
416/416 [==============================] - 1s 2ms/step - loss: 0.0874
Epoch 8/1000
416/416 [==============================] - 1s 2ms/step - loss: 0.0761
Epoch 9/1000
416/416 [==============================] - 1s 3ms/step - loss: 0.0672
Epoch 10/1000
416/416 [==============================] - 1s 2ms/step - loss: 0.0796
Epoch 11/1000
416/416 [==============================] - 1s 3ms/step - loss: 0.0672
Epoch 12/1000
416/416 [==============================] - 1s 3ms/step - loss: 0.0917
E

### Save Autoencoder Model

In [9]:
ae_model.save('./habitech_autoencoder')
ae_model.save('./habitech_autoencoder.h5')
ae_model.save_weights("./habitech_autoencoder_weights.h5")

INFO:tensorflow:Assets written to: ./habitech_autoencoder/assets


### Embed Data using Autoencoder Model

In [10]:
hl = ae_model.predict(tensor_df)

### Build Manifold Learner from Embedded Data

In [11]:
hle = manifold_learner()
hle_result = hle.fit_transform(hl)

### Save Manifold Learner (UMAP) Model

In [12]:
import joblib

joblib.dump(hle, 'habitech_umap.sav')

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


['habitech_umap.sav']

### Build Clustering Model Using Data that has been Processed by UMAP

In [13]:
gmm = clustering_algorithm()
gmm_result = gmm.fit(hle_result)

### Save Manifold Learner (UMAP) Model

In [14]:
joblib.dump(gmm, 'habitech_gmm.sav')

['habitech_gmm.sav']

### Predict Cluster Probability of Each Data Using GMM Model that has been Built

In [15]:
y_pred_prob = gmm_result.predict_proba(hle_result)

### Get Maximum Value Index from Each Data (Assigned Cluster for User)

In [16]:
y_pred = y_pred_prob.argmax(1)

In [17]:
y_pred

array([ 6, 32,  4, ...,  2, 19,  7])

## (Optional) Download Files from Google Colab

In [18]:
from google.colab import files

files.download('./habitech_autoencoder.h5')
files.download('./habitech_autoencoder_weights.h5')
files.download('./habitech_gmm.sav')
files.download('./habitech_umap.sav')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
!zip -r /content/habitech_autoencoder.zip /content/habitech_autoencoder

files.download('./habitech_autoencoder.zip')

  adding: content/habitech_autoencoder/ (stored 0%)
  adding: content/habitech_autoencoder/saved_model.pb (deflated 90%)
  adding: content/habitech_autoencoder/variables/ (stored 0%)
  adding: content/habitech_autoencoder/variables/variables.data-00000-of-00001 (deflated 69%)
  adding: content/habitech_autoencoder/variables/variables.index (deflated 69%)
  adding: content/habitech_autoencoder/keras_metadata.pb (deflated 92%)
  adding: content/habitech_autoencoder/assets/ (stored 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>